# DrainageAI Demo - Google Colab Integration

This notebook demonstrates the DrainageAI super-MVP workflow using Google Colab's GPU acceleration. It allows you to detect drainage pipes in agricultural fields using satellite imagery and spectral indices.

## Workflow Overview
1. Check GPU availability
2. Install dependencies
3. Clone the DrainageAI repository
4. Upload test imagery
5. Calculate spectral indices
6. Detect drainage pipes
7. Vectorize results
8. Visualize and download results

## Step 1: Check GPU Availability

First, let's check if a GPU is available for acceleration.

In [ ]:
import torch
print(f"GPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU name: {torch.cuda.get_device_name(0)}")
else:
    print("WARNING: No GPU detected. Processing will be slow.")

## Step 2: Install Dependencies

Install the required packages for DrainageAI.

In [ ]:
!pip install rasterio geopandas scikit-image matplotlib pytorch-lightning torch-geometric

## Step 3: Clone the DrainageAI Repository

Clone the repository and navigate to its directory.

In [ ]:
# Clone the repository
!git clone https://github.com/yourusername/DrainageAI.git
%cd DrainageAI

## Step 4: Upload Test Imagery

Upload your multispectral imagery file (GeoTIFF format).

In [ ]:
from google.colab import files
print("Please upload your multispectral imagery file (GeoTIFF format):")
uploaded = files.upload()

# Get the filename of the uploaded file
imagery_filename = list(uploaded.keys())[0]
print(f"Uploaded file: {imagery_filename}")

## Step 5: Create Output Directory

In [ ]:
!mkdir -p colab_results

## Step 6: Calculate Spectral Indices

Calculate NDVI, NDMI, and MSAVI2 from the multispectral imagery.

In [ ]:
print("\n=== Step 1: Calculate Spectral Indices ===\n")
!python main.py indices --imagery {imagery_filename} --output colab_results/indices.tif --indices ndvi,ndmi,msavi2

## Step 7: Run Drainage Detection

Detect drainage pipes using the semi-supervised model with spectral indices.

In [ ]:
print("\n=== Step 2: Detect Drainage Pipes ===\n")
!python main.py detect --imagery {imagery_filename} --indices colab_results/indices.tif --output colab_results/drainage.tif --model semi

## Step 8: Vectorize Results

Convert the raster detection results to vector format for GIS analysis.

In [ ]:
print("\n=== Step 3: Vectorize Results ===\n")
!python main.py vectorize --input colab_results/drainage.tif --output colab_results/drainage_lines.shp

## Step 9: Visualize Results

Create visualizations of the original imagery, spectral indices, and detection results.

In [ ]:
import matplotlib.pyplot as plt
import rasterio
import numpy as np
import geopandas as gpd
from rasterio.plot import show

# Load original imagery
with rasterio.open(imagery_filename) as src:
    # Create RGB composite (assuming bands are in order)
    rgb = np.dstack((src.read(3), src.read(2), src.read(1)))  # R,G,B
    # Normalize for display
    rgb = (rgb - rgb.min()) / (rgb.max() - rgb.min())
    
    # Get metadata for plotting
    extent = [src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top]

# Load indices
with rasterio.open("colab_results/indices.tif") as src:
    ndvi = src.read(1)  # Assuming NDVI is the first band
    ndmi = src.read(2) if src.count > 1 else None  # NDMI if available
    
# Load detection results
with rasterio.open("colab_results/drainage.tif") as src:
    detection = src.read(1)

# Load vectorized results
try:
    drainage_lines = gpd.read_file("colab_results/drainage_lines.shp")
    has_vectors = True
except Exception as e:
    has_vectors = False
    print(f"No vector results found or error loading them: {e}")

# Create visualization
plt.figure(figsize=(20, 15))

# Original RGB
plt.subplot(221)
plt.imshow(rgb)
plt.title("Original RGB Image")
plt.axis('off')

# NDVI
plt.subplot(222)
plt.imshow(ndvi, cmap='RdYlGn')
plt.title("NDVI")
plt.colorbar(shrink=0.5)
plt.axis('off')

# Detection result
plt.subplot(223)
plt.imshow(rgb)
if has_vectors:
    drainage_lines.plot(ax=plt.gca(), color='red', linewidth=1)
plt.title("Drainage Lines Overlay")
plt.axis('off')

# Detection probability
plt.subplot(224)
plt.imshow(detection, cmap='jet')
plt.title("Drainage Detection Probability")
plt.colorbar(shrink=0.5)
plt.axis('off')

plt.tight_layout()
plt.show()

## Step 10: Download Results

Download the results files for use in ArcGIS or QGIS.

In [ ]:
print("\nDownload the results files:")
files.download("colab_results/drainage_lines.shp")
files.download("colab_results/drainage_lines.shx")
files.download("colab_results/drainage_lines.dbf")
files.download("colab_results/drainage_lines.prj")
files.download("colab_results/drainage.tif")
files.download("colab_results/indices.tif")

print("\nWorkflow complete! You can now load these files in ArcGIS or QGIS.")

## Additional Options

### Adjusting Detection Threshold

If the initial results don't show enough drainage features or show too many false positives, you can adjust the threshold:

In [ ]:
# Example with lower threshold (more sensitive)
# !python main.py detect --imagery {imagery_filename} --indices colab_results/indices.tif --output colab_results/drainage_sensitive.tif --model semi --threshold 0.3

# Example with higher threshold (less sensitive)
# !python main.py detect --imagery {imagery_filename} --indices colab_results/indices.tif --output colab_results/drainage_specific.tif --model semi --threshold 0.7

### Using Different Models

You can also try different models:

In [ ]:
# Example with CNN model
# !python main.py detect --imagery {imagery_filename} --indices colab_results/indices.tif --output colab_results/drainage_cnn.tif --model cnn

# Example with ensemble model
# !python main.py detect --imagery {imagery_filename} --indices colab_results/indices.tif --output colab_results/drainage_ensemble.tif --model ensemble